In [20]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
delaney_chemberta2_valid2 = pd.read_csv('./chemberta2/results/delaney/chemberta2_valid2_delaney_2_predictions.csv')
delaney_molformer_valid2 = pd.read_csv('./molformer/results/delaney/molformer_valid2_delaney_2_99.csv')
delaney_molbert_valid2 = pd.read_csv('./molbert/results/delaney/molbert_valid2_delaney_2.csv')

# Load the test data for each model
delaney_chemberta2_test = pd.read_csv('./chemberta2/results/delaney/chemberta2_test_delaney_2_predictions.csv')
delaney_molformer_test = pd.read_csv('./molformer/results/delaney/molformer_test_delaney_2_99.csv')
delaney_molbert_test = pd.read_csv('./molbert/results/delaney/molbert_test_delaney_2.csv')

train_mean = -3.0330665680473374
train_sd = 2.0662151693210706

# features

# Load the features from chemberta
delaney_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/delaney/chemberta2_valid2_delaney_2_features.csv')
delaney_chemberta2_features_test = pd.read_csv('./chemberta2/features/delaney/chemberta2_test_delaney_2_features.csv')

# Load the features from molformer
delaney_molformer_features_valid2 = pd.read_csv('./molformer/features/delaney/molformer_valid2_delaney_2_features.csv')
delaney_molformer_features_test = pd.read_csv('./molformer/features/delaney/molformer_test_delaney_2_features.csv')

# Load the features from molbert
delaney_molbert_features_valid2 = pd.read_csv('./molbert/features/delaney/molbert_valid2_delaney_2_features.csv')
delaney_molbert_features_test = pd.read_csv('./molbert/features/delaney/molbert_test_delaney_2_features.csv')

For delaney (regression)

In [21]:
# Preparing the actual and predicted values
# Chemberta2
delaney_chemberta_actual = delaney_chemberta2_test['target'] 
delaney_chemberta_pred = delaney_chemberta2_test['pred_raw']

# Molformer
delaney_molformer_actual = delaney_molformer_test['target']
delaney_molformer_pred = delaney_molformer_test['pred_raw']

# molbert
delaney_molbert_actual = delaney_molbert_test['target_raw']
delaney_molbert_pred = delaney_molbert_test['pred_raw']

In [22]:
# Calculating metrics
delaney_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", delaney_chemberta_actual, delaney_chemberta_pred),
                                 ("Molformer", delaney_molformer_actual, delaney_molformer_pred),
                                 ("Molbert", delaney_molbert_actual, delaney_molbert_pred)]:
    delaney_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

delaney_metrics_results

{'Chemberta2': {'MAE': 0.4969548122463181,
  'RMSE': 0.6739879851585397,
  'R2 Score': 0.8995271768026132,
  'Correlation': 0.9520502440343135},
 'Molformer': {'MAE': 0.47515824184070793,
  'RMSE': 0.6199320972655071,
  'R2 Score': 0.9149973350094271,
  'Correlation': 0.9664923764380632},
 'Molbert': {'MAE': 0.39575742491150445,
  'RMSE': 0.5147120546385228,
  'R2 Score': 0.9414033246688129,
  'Correlation': 0.9711342185240168}}

In [23]:
# standardized valid2 labels
delaney_y_ensemble_valid2 = (delaney_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
delaney_X_ensemble_valid2 = pd.concat([
    delaney_chemberta2_valid2['pred_z'] - delaney_y_ensemble_valid2,
    delaney_molformer_valid2['pred_z'] - delaney_y_ensemble_valid2, 
    delaney_molbert_valid2['pred_z'] - delaney_y_ensemble_valid2,
    # add features from training set
    delaney_chemberta2_features_valid2.iloc[:, 2:],
    delaney_molformer_features_valid2.iloc[:, 1:],
    delaney_molbert_features_valid2.iloc[:, 1:]
], axis=1)

# change feature names of the ensemble so that they are unique
delaney_X_ensemble_valid2.columns = ['chemberta', 'molformer', 'molbert'] + list(delaney_chemberta2_features_valid2.columns[2:]) + list(delaney_molformer_features_valid2.columns[1:]) + list(delaney_molbert_features_valid2.columns[1:])

# standardized test labels
delaney_y_ensemble_test_std = (delaney_chemberta2_test['target']  - train_mean)/train_sd

delaney_X_ensemble_test = pd.concat([
    delaney_chemberta2_test['pred_z'] - delaney_y_ensemble_test_std,
    delaney_molformer_test['pred_z'] - delaney_y_ensemble_test_std,  
    delaney_molbert_test['pred_z'] - delaney_y_ensemble_test_std,
    # add features from test set
    delaney_chemberta2_features_test.iloc[:, 2:],
    delaney_molformer_features_test.iloc[:, 1:],
    delaney_molbert_features_test.iloc[:, 1:]
], axis=1)

# change feature names of the ensemble so that they are unique
delaney_X_ensemble_test.columns = ['chemberta', 'molformer', 'molbert'] + list(delaney_chemberta2_features_test.columns[2:]) + list(delaney_molformer_features_test.columns[1:]) + list(delaney_molbert_features_test.columns[1:])

# true test labels
delaney_y_ensemble_test = delaney_chemberta2_test['target']

In [24]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
delaney_X_ensemble_valid2_scaled = scaler.fit_transform(delaney_X_ensemble_valid2)
delaney_X_ensemble_test_scaled = scaler.transform(delaney_X_ensemble_test)

delaney_X_ensemble_valid2_scaled = pd.DataFrame(delaney_X_ensemble_valid2_scaled, columns=delaney_X_ensemble_valid2.columns)
delaney_X_ensemble_test_scaled = pd.DataFrame(delaney_X_ensemble_test_scaled, columns=delaney_X_ensemble_test.columns)

In [25]:
# export delaney_X_ensemble_valid2 and delaney_y_ensemble_valid2 to csv
delaney_X_ensemble_valid2_scaled.to_csv('./processed_data/delaney_X_ensemble_valid2_scaled_rawres.csv', index=False)
delaney_X_ensemble_test_scaled.to_csv('./processed_data/delaney_X_ensemble_test_scaled_rawres.csv', index=False)

# export delaney_X_ensemble_test and delaney_y_ensemble_test to csv
delaney_y_ensemble_valid2.to_csv('./processed_data/delaney_y_ensemble_valid2.csv', index=False)
delaney_y_ensemble_test.to_csv('./processed_data/delaney_y_ensemble_test.csv', index=False)

In [6]:
# lasso model
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the LassoCV model
lasso_cv = LassoCV(cv=5, max_iter = 5000, random_state=0)

# Fit the model
lasso_cv.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Predict the test set
delaney_lasso_pred = lasso_cv.predict(delaney_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the metrics
delaney_lasso_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_lasso_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_lasso_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_lasso_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_lasso_pred)[0]
}

delaney_lasso_metrics

{'MAE': 0.4753863884431177,
 'RMSE': 0.6284275900541463,
 'R2 Score': 0.9126516342538643,
 'Correlation': 0.955493686085796}

In [7]:
# use lasso to select features
coefs = pd.Series(lasso_cv.coef_, index=delaney_X_ensemble_valid2.columns)

selected_features = coefs[coefs != 0].index.tolist()

print("Selected Features:", selected_features)

chemberta_count = sum('chemberta' in feature for feature in selected_features)
molformer_count = sum('molformer' in feature for feature in selected_features)
molbert_count = sum('molbert' in feature for feature in selected_features)

print("Chemberta2 count:", chemberta_count)
print("Molformer count:", molformer_count)
print("Molbert count:", molbert_count)

Selected Features: ['chemberta', 'molformer', 'molbert', 'chemberta2_feature_2', 'chemberta2_feature_20', 'chemberta2_feature_28', 'chemberta2_feature_38', 'chemberta2_feature_51', 'chemberta2_feature_81', 'chemberta2_feature_123', 'chemberta2_feature_137', 'chemberta2_feature_139', 'chemberta2_feature_169', 'chemberta2_feature_171', 'chemberta2_feature_180', 'chemberta2_feature_186', 'chemberta2_feature_215', 'chemberta2_feature_228', 'chemberta2_feature_234', 'chemberta2_feature_239', 'chemberta2_feature_271', 'chemberta2_feature_284', 'chemberta2_feature_308', 'chemberta2_feature_318', 'chemberta2_feature_320', 'chemberta2_feature_322', 'chemberta2_feature_346', 'chemberta2_feature_351', 'chemberta2_feature_381', 'molformer_feature_4', 'molformer_feature_10', 'molformer_feature_11', 'molformer_feature_12', 'molformer_feature_15', 'molformer_feature_20', 'molformer_feature_37', 'molformer_feature_55', 'molformer_feature_76', 'molformer_feature_91', 'molformer_feature_98', 'molformer_

In [8]:
import numpy as np
from skglm import GroupLasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# two groups: one for the prediction probabilities and one for the features

# Define the groups for each feature
n_features = delaney_X_ensemble_valid2_scaled.shape[1]

groups = [
    list(range(0, 3)),  # Group 0 with feature indices 0, 1, 2
    list(range(3, n_features))  # Group 1 with all remaining features
]

# Initialize the GroupLasso model
group_lasso_model = GroupLasso(
    groups=groups,
    alpha=1.0,
    p0=10,
    verbose=0,
    tol=0.0001,
    positive=False,
    fit_intercept=True,
    warm_start=False,
)


# Setup cross-validation to find the best alpha
param_grid = {'alpha': np.logspace(-4, 1, 10)}
cv = GridSearchCV(
    estimator=group_lasso_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5
)

# Fit GridSearchCV
cv.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Best model and parameters
best_model = cv.best_estimator_
print("Best alpha:", cv.best_params_['alpha'])

# Predict using the best model
delaney_pred = best_model.predict(delaney_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the evaluation metrics
delaney_two_groups_lasso_best_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_pred)[0]  # Only record the correlation coefficient
}

# Print the calculated metrics
delaney_two_groups_lasso_best_metrics

Best alpha: 0.0001


{'MAE': 0.47171324101554574,
 'RMSE': 0.5878552441343862,
 'R2 Score': 0.9235662636386849,
 'Correlation': 0.9611639547835964}

In [9]:
# four groups: predictions, features from chemberta, features from molformer, features from molbert

# Define the groups for each feature
n_features = delaney_X_ensemble_valid2_scaled.shape[1]
groups = [
    list(range(0, 3)),  # Group 0 with feature indices 0, 1, 2
    list(range(3, 3 + 384)),  # Group 1 with next 384 features
    list(range(3 + 384, 3 + 384 + 768)),  # Group 2 with next 768 features
    list(range(3 + 384 + 768, n_features))  # Group 3 with all remaining features
]

# Initialize the GroupLasso model
group_lasso_model = GroupLasso(
    groups=groups,
    alpha=1.0,
    p0=10,
    verbose=0,
    tol=0.0001,
    positive=False,
    fit_intercept=True,
    warm_start=False,
)


# Setup cross-validation to find the best alpha
param_grid = {'alpha': np.logspace(-4, 1, 10)}
cv = GridSearchCV(
    estimator=group_lasso_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5
)

# Fit GridSearchCV
cv.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Best model and parameters
best_model = cv.best_estimator_
print("Best alpha:", cv.best_params_['alpha'])

# Predict using the best model
delaney_pred = best_model.predict(delaney_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the evaluation metrics
delaney_four_groups_lasso_best_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_pred)[0]  # Only record the correlation coefficient
}

# Print the calculated metrics
delaney_four_groups_lasso_best_metrics

Best alpha: 0.0001


{'MAE': 0.45465258075971926,
 'RMSE': 0.5898495901276928,
 'R2 Score': 0.9230467687735883,
 'Correlation': 0.96101616603471}

In [10]:
# elastic net
# Define the model with elasticnet penalty for regression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

elastic_net_model = ElasticNet(random_state=0, max_iter=5000)

# Define the hyperparameter grid
# Use fewer discrete values for alpha and l1_ratio
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range
l1_ratios = [0.1, 0.5, 0.9]  # Reduced to three points, emphasizing edges and midpoint

params = {
    'alpha': alphas,  # Convert alpha back to C
    'l1_ratio': l1_ratios
}

grid_search = GridSearchCV(elastic_net_model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Get the best hyperparameters
delaney_best_elastic_params = grid_search.best_params_
print(delaney_best_elastic_params)

# Initialize and train the best ElasticNet model
delaney_best_elastic_model = ElasticNet(alpha=delaney_best_elastic_params['alpha'], l1_ratio=delaney_best_elastic_params['l1_ratio'], random_state=0, max_iter=5000)
delaney_best_elastic_model.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Predict the test set
delaney_elastic_pred = delaney_best_elastic_model.predict(delaney_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the metrics
delaney_elastic_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_elastic_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_elastic_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_elastic_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_elastic_pred)[0]
}

delaney_elastic_metrics

{'alpha': 0.1, 'l1_ratio': 0.1}


{'MAE': 0.45847888068493553,
 'RMSE': 0.5856868902509849,
 'R2 Score': 0.9241290883347271,
 'Correlation': 0.9615139360608405}

In [11]:
# use elastic net to select features
coefs = pd.Series(delaney_best_elastic_model.coef_, index=delaney_X_ensemble_valid2.columns)

selected_features = coefs[coefs != 0].index.tolist()

print("Selected Features:", selected_features)

chemberta_count = sum('chemberta' in feature for feature in selected_features)
molformer_count = sum('molformer' in feature for feature in selected_features)
molbert_count = sum('molbert' in feature for feature in selected_features)

print("Chemberta2 count:", chemberta_count)
print("Molformer count:", molformer_count)
print("Molbert count:", molbert_count)

Selected Features: ['chemberta', 'molformer', 'molbert', 'chemberta2_feature_2', 'chemberta2_feature_20', 'chemberta2_feature_28', 'chemberta2_feature_38', 'chemberta2_feature_42', 'chemberta2_feature_51', 'chemberta2_feature_63', 'chemberta2_feature_81', 'chemberta2_feature_98', 'chemberta2_feature_123', 'chemberta2_feature_137', 'chemberta2_feature_139', 'chemberta2_feature_169', 'chemberta2_feature_171', 'chemberta2_feature_172', 'chemberta2_feature_179', 'chemberta2_feature_180', 'chemberta2_feature_186', 'chemberta2_feature_215', 'chemberta2_feature_228', 'chemberta2_feature_234', 'chemberta2_feature_239', 'chemberta2_feature_242', 'chemberta2_feature_271', 'chemberta2_feature_282', 'chemberta2_feature_284', 'chemberta2_feature_301', 'chemberta2_feature_308', 'chemberta2_feature_313', 'chemberta2_feature_318', 'chemberta2_feature_320', 'chemberta2_feature_322', 'chemberta2_feature_331', 'chemberta2_feature_337', 'chemberta2_feature_346', 'chemberta2_feature_347', 'chemberta2_featu

In [12]:
delaney_X_ensemble_valid2_selected = delaney_X_ensemble_valid2_scaled
delaney_X_ensemble_test_selected = delaney_X_ensemble_test_scaled

# check shapes
print(delaney_X_ensemble_valid2_selected.shape)
print(delaney_X_ensemble_test_selected.shape)

(226, 1923)
(113, 1923)


In [13]:
# Initialize and train the SVR model
from sklearn.svm import SVR

delaney_svr_model = SVR()
delaney_svr_model.fit(delaney_X_ensemble_valid2_selected, delaney_y_ensemble_valid2)

# Predict the test set
delaney_svr_pred = delaney_svr_model.predict(delaney_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
delaney_svr_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_svr_pred ),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_svr_pred )),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_svr_pred ),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_svr_pred )[0]  # Only record the correlation coefficient
}

delaney_svr_metrics

{'MAE': 0.6023343979283677,
 'RMSE': 0.8152583534886683,
 'R2 Score': 0.8529940844020749,
 'Correlation': 0.9422536036552377}

In [14]:
# initailize and use a 5-fold cross-validation to tune the hyperparameters of a random forest model for regression
from sklearn.ensemble import RandomForestRegressor

delaney_rf_model = RandomForestRegressor(random_state=0)

delaney_rf_model.fit(delaney_X_ensemble_valid2_selected, delaney_y_ensemble_valid2)

# Predict the test set
delaney_rf_best_pred = delaney_rf_model.predict(delaney_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
delaney_rf_best_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_rf_best_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_rf_best_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_rf_best_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_rf_best_pred)[0]  # Only record the correlation coefficient
}

delaney_rf_best_metrics

{'MAE': 0.6499758407079647,
 'RMSE': 0.8336024127955629,
 'R2 Score': 0.846304121236927,
 'Correlation': 0.9332187335067594}

In [15]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Define the hyperparameter space using continuous distributions
delaney_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Correctly define the RMSE scorer function
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    model = xgb.XGBRegressor(**params, random_state=0)
    
    # Cross-validated RMSE as the objective
    score = cross_val_score(model, delaney_X_ensemble_valid2_selected, delaney_y_ensemble_valid2, 
                            scoring=make_scorer(rmse_scorer, greater_is_better=False), cv=5)
    
    # Minimize the positive RMSE (already negative from scoring)
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
delaney_xgb_best_params = fmin(fn=objective, 
                            space=delaney_xgb_hyperopt_space, 
                            algo=tpe.suggest, 
                            max_evals=100, 
                            trials=trials,
                            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", delaney_xgb_best_params)


 13%|█▎        | 13/100 [02:53<19:23, 13.38s/trial, best loss: 0.4446807052857352]
Best hyperparameters: {'colsample_bytree': 0.9744918324309589, 'learning_rate': 0.16784189395268395, 'max_depth': 4.0, 'n_estimators': 200.0, 'subsample': 0.633963070790543}


In [16]:
# fit the model with the best hyperparameters
# Convert parameters obtained from Hyperopt to the correct data type
delaney_xgb_best_params['n_estimators'] = int(delaney_xgb_best_params['n_estimators'])
delaney_xgb_best_params['max_depth'] = int(delaney_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
delaney_xgb_model = xgb.XGBRegressor(**delaney_xgb_best_params, random_state=0)
delaney_xgb_model.fit(delaney_X_ensemble_valid2_selected, delaney_y_ensemble_valid2)

# Predict the test set
delaney_xgb_best_pred = delaney_xgb_model.predict(delaney_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
delaney_xgb_best_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_xgb_best_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_xgb_best_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_xgb_best_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_xgb_best_pred)[0]  # Only record the correlation coefficient
}

delaney_xgb_best_metrics

{'MAE': 0.5691241254131351,
 'RMSE': 0.7307004335561956,
 'R2 Score': 0.8819073069866527,
 'Correlation': 0.9469492120818824}

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

torch.manual_seed(0)

# Define RMSE loss
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model for regression
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Hyperparameter space with hp.quniform for integer distribution
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Global dataset variables assumed to be defined externally
X = delaney_X_ensemble_valid2_selected
y = delaney_y_ensemble_valid2

# Objective function for Bayesian optimization
def objective(params):
    params['num_layers'] = int(params['num_layers'])  # Ensure num_layers is an integer
    params['num_neurons'] = int(params['num_neurons'])  # Ensure num_neurons is an integer
    kf = KFold(n_splits=5)
    rmse_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Convert DataFrame to numpy arrays before making them PyTorch tensors
        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=params['num_layers'],
                         num_neurons=params['num_neurons'], dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(torch.tensor(X_val.values.astype(np.float32))).squeeze(1)
            val_targets = torch.tensor(y_val.values.astype(np.float32))
            rmse = np.sqrt(mean_squared_error(val_targets.numpy(), val_preds.numpy()))
            rmse_scores.append(rmse)

    avg_rmse = np.mean(rmse_scores)
    return {'loss': avg_rmse, 'status': STATUS_OK} # Minimize RMSE

# Run Bayesian optimization
trials = Trials()
delaney_nn_best_params = fmin(fn=objective,
                           space=space,
                           algo=tpe.suggest,
                           max_evals=50,
                           trials=trials,
                           early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", delaney_nn_best_params)


 28%|██▊       | 14/50 [00:54<02:20,  3.90s/trial, best loss: 0.38242167234420776]
Best hyperparameters: {'dropout_rate': 0.30356752647384105, 'learning_rate': 0.0004090869358078568, 'num_layers': 1.0, 'num_neurons': 74.0}


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Define a function to compute RMSE
def compute_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert parameters to the correct format if necessary
delaney_nn_best_params = {
    'num_layers':  int(delaney_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons':  int(delaney_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': delaney_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': delaney_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(delaney_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(delaney_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(delaney_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(delaney_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=delaney_X_ensemble_valid2_selected.shape[1], num_layers=delaney_nn_best_params['num_layers'],
                         num_neurons=delaney_nn_best_params['num_neurons'], dropout_rate=delaney_nn_best_params['dropout_rate'])
criterion = RMSELoss()
optimizer = optim.Adam(model.parameters(), lr=delaney_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = outputs.squeeze(1).numpy() * train_sd + train_mean

    # Calculate metrics
    mae = mean_absolute_error(y_test_tensor.numpy(), predictions)
    rmse = compute_rmse(y_test_tensor.numpy(), predictions)
    r2 = r2_score(y_test_tensor.numpy(), predictions)
    correlation, _ = pearsonr(y_test_tensor.numpy().squeeze(1), predictions)

    delaney_nn_metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'R2 Score': r2,
        'Correlation': correlation
    }

delaney_nn_metrics

{'MAE': 0.4630387,
 'RMSE': 0.60733396,
 'R2 Score': 0.9184170497817512,
 'Correlation': 0.9608877759777632}

In [19]:
# create a table to record all metrics for delaney
delaney_metrics_results["LASSO"] = delaney_lasso_metrics
delaney_metrics_results["Group Lasso (Two Groups)"] = delaney_two_groups_lasso_best_metrics
delaney_metrics_results["Group Lasso (Four Groups)"] = delaney_four_groups_lasso_best_metrics
delaney_metrics_results["Elastic Net"] = delaney_elastic_metrics
delaney_metrics_results["SVR"] = delaney_svr_metrics
delaney_metrics_results["Random Forest"] = delaney_rf_best_metrics
delaney_metrics_results["XGBoost"] = delaney_xgb_best_metrics
delaney_metrics_results["Neural Network"] = delaney_nn_metrics

delaney_metrics_df = pd.DataFrame(delaney_metrics_results).T
# keep 3 digits after the decimal point
delaney_metrics_df = delaney_metrics_df.round(3)

# export table to csv
delaney_metrics_df.to_csv('./split2_delaney_metrics_rawres.csv')